In [49]:
import pandas as pd
import plotly.express as px


In [69]:
exp = None

# Concatenate expenses dataframes
for filename in ["expenses", "expenses_new", "expenses3", "expenses_saving"]:
  df = pd.read_csv(f"../data/raw/{filename}.csv", sep=";", header=3, decimal=",")
  if exp is None:
    exp = df
  else:
    exp = pd.concat([exp, df], ignore_index=True, verify_integrity=True)
    exp = exp.drop_duplicates()

exp["Date d'opération"] = pd.to_datetime(exp["Date d'opération"], dayfirst=True)
exp = exp.sort_values("Date d'opération", ignore_index=True)


In [107]:
exp_num = exp.groupby("Date d'opération").sum(numeric_only=True).reset_index()

# Count operations since start of trading
mask_since_trading = exp_num["Date d'opération"] > "2021-09-01"
num_operations_since_trading = (mask_since_trading).sum()
traded_value = 10000

# Compute average traded value w.r.t. num operations
traded_value_per_operation = traded_value / num_operations_since_trading

# Add average traded value
exp_num.loc[mask_since_trading, "Montant"] += traded_value_per_operation

exp_num["cumsum"] = exp_num["Montant"].cumsum()


In [108]:
px.line(exp_num, x="Date d'opération", y="cumsum")
